In [1]:
import pinecone
from sentence_transformers import SentenceTransformer
from groq import Groq
from typing import List, Dict
import os
from dotenv import load_dotenv

/Users/ismaelrobles-razzaq/cs_projects/venv1/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
class RSORagBot:
    def __init__(self, pinecone_api_key: str, pinecone_index_name: str, groq_api_key: str):
        # Initialize Pinecone
        self.pc = pinecone.Pinecone(api_key=pinecone_api_key)
        self.index = self.pc.Index(pinecone_index_name)
        
        # Initialize embedding model
        self.embed_model = SentenceTransformer('all-mpnet-base-v2')
        
        # Initialize Groq client
        self.groq_client = Groq(api_key=groq_api_key)
        
        # Define system prompt
        self.system_prompt = """You are a helpful assistant for University of Chicago students, 
        helping them find Registered Student Organizations (RSOs) that match their interests. 
        Use the provided RSO information to give detailed, relevant recommendations. 
        Always include the RSO's name, a brief description, and contact information when available."""

    def get_relevant_rsos(self, query: str, top_k: int = 5) -> List[Dict]:
        """
        Get relevant RSOs based on the query
        """
        # Create query embedding
        query_embedding = self.embed_model.encode(query).tolist()
        
        # Query Pinecone
        results = self.index.query(
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True
        )
        
        return results.matches

    def format_context(self, relevant_rsos: List[Dict]) -> str:
        """
        Format RSO information into a context string for the LLM
        """
        context = "Here are some relevant RSOs:\n\n"
        
        for rso in relevant_rsos:
            metadata = rso.metadata
            context += f"Name: {metadata['name']}\n"
            context += f"Description: {metadata['description']}\n"
            context += f"Categories: {', '.join(metadata['categories'])}\n"
            context += f"Contact: {metadata['contact_email']}\n"
            if metadata['social_media_links']:
                context += f"Social Media: {', '.join(metadata['social_media_links'])}\n"
            if metadata['additional_info']:
                context += f"Additional Info: {', '.join(metadata['additional_info'])}\n"
            context += f"Website: {metadata['full_url']}\n\n"
        
        return context

    def generate_response(self, query: str) -> str:
        """
        Generate a response to the user's query
        """
        # Get relevant RSOs
        relevant_rsos = self.get_relevant_rsos(query)
        
        # Format context
        context = self.format_context(relevant_rsos)
        
        # Construct the prompt
        prompt = f"""Based on the following RSO information, please recommend appropriate RSOs for a student with this query: "{query}"

        {context}

        Please provide a natural, conversational response that:
        1. Highlights the most relevant RSOs for their interests
        2. Explains why each RSO might be a good fit
        3. Includes practical information like how to get involved
        """
        
        # Get response from Groq
        response = self.groq_client.chat.completions.create(
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt}
            ],
            model="mixtral-8x7b-32768",  # or your preferred Groq model
            temperature=0.7,
            max_tokens=1024
        )
        
        return response.choices[0].message.content

In [3]:
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")


In [4]:
bot = RSORagBot(
        pinecone_api_key=os.getenv("PINECONE_API_KEY"),
        pinecone_index_name="rso-chatbot",  # replace with your index name
        groq_api_key=os.getenv("GROQ_API_KEY")
    )
    
    

In [5]:
# Example query

query = "what's your rso question?"
response = bot.generate_response(query)
print(response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Hello! It's great to hear that you're interested in getting involved with Registered Student Organizations (RSOs) at the University of Chicago. Based on your interest in RSOs, I have a few suggestions that might align well with your preferences.

1. Rural Student Alliance (RSA)
   The Rural Student Alliance (RSA) is a wonderful community for students who come from rural areas or are interested in learning more about the rural experience. RSA aims to create a supportive space for rural students while educating the broader community. They offer community service and volunteering opportunities, and cater to cultural interests. You can get in touch with them by emailing htroupe@uchicago.edu or joining their mailing list, rural@lists.uchicago.edu. You can also follow them on Instagram (@uchicagorsa) or Facebook (UChicagoRSA) for updates on their events and activities.

2. Engineering Society
   If you have an academic interest in engineering or enjoy working on large-scale engineering proje